In [1]:
import os

distances = ["one meter", "two meter", "three meter", "four meter"]
prompts = ["a man walks straight forward for *", "a man walks in a straight line for *", "a man runs straight forward for *", "a man runs in a straight line for *" ]
for p in prompts:
    for d in distances: 
        if p.split('*')[1] == "":
            space_ornot = ""
        else :
            space_ornot = " "
        print("python -m sample.generate --model_path ./save/humanml_trans_enc_512/model000200000.pt --text_prompt " + p.split('*')[0] + str(d) + space_ornot + p.split('*')[1] + ".")

python -m sample.generate --model_path ./save/humanml_trans_enc_512/model000200000.pt --text_prompt a man walks straight forward for one meter.
python -m sample.generate --model_path ./save/humanml_trans_enc_512/model000200000.pt --text_prompt a man walks straight forward for two meter.
python -m sample.generate --model_path ./save/humanml_trans_enc_512/model000200000.pt --text_prompt a man walks straight forward for three meter.
python -m sample.generate --model_path ./save/humanml_trans_enc_512/model000200000.pt --text_prompt a man walks straight forward for four meter.
python -m sample.generate --model_path ./save/humanml_trans_enc_512/model000200000.pt --text_prompt a man walks in a straight line for one meter.
python -m sample.generate --model_path ./save/humanml_trans_enc_512/model000200000.pt --text_prompt a man walks in a straight line for two meter.
python -m sample.generate --model_path ./save/humanml_trans_enc_512/model000200000.pt --text_prompt a man walks in a straight lin

In [4]:
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation, FFMpegFileWriter
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import mpl_toolkits.mplot3d.axes3d as p3
import mpl_toolkits.mplot3d.art3d as art3d

from matplotlib.patches import Circle

# import cv2
from textwrap import wrap


def list_cut_average(ll, intervals):
    if intervals == 1:
        return ll

    bins = math.ceil(len(ll) * 1.0 / intervals)
    ll_new = []
    for i in range(bins):
        l_low = intervals * i
        l_high = l_low + intervals
        l_high = l_high if l_high < len(ll) else len(ll)
        ll_new.append(np.mean(ll[l_low:l_high]))
    return ll_new


def plot_3d_motion(save_path, kinematic_tree, joints, title, dataset, figsize=(3, 3), fps=120, radius=3,
                   vis_mode='default', gt_frames=[]):
    matplotlib.use('Agg')

    title = '\n'.join(wrap(title, 20))

    def init():
        #print("radius", radius)
        ax.set_xlim3d([-radius / 2, radius / 2])
        ax.set_ylim3d([0, radius])
        ax.set_zlim3d([-radius / 3., radius * 2 / 3.])
        # print(title)
        
        
        
        title = 'a man is pretending to be a chicken. constantly pecking at the ground and waving his arms like a chicken.'
        
        
        
        fig.suptitle(title, fontsize=10)
        ax.grid(b=False)

    def plot_xzPlane(minx, maxx, miny, minz, maxz):
        ## Plot a plane XZ
        verts = [
            [minx, miny, minz],
            [minx, miny, maxz],
            [maxx, miny, maxz],
            [maxx, miny, minz]
        ]
        #print("verts", verts)
        xz_plane = Poly3DCollection([verts])
        xz_plane.set_facecolor((0.5, 0.5, 0.5, 0.5))
        ax.add_collection3d(xz_plane)

    def plot_xzPlane2(minx, maxx, miny, minz, maxz):
        ## Plot a plane XZ
        verts = [
            [minx, miny, minz],
            [minx, miny, maxz],
            [maxx, miny, maxz],
            [maxx, miny, minz]
        ]
        #print("verts", verts)
        xz_plane2 = Poly3DCollection([verts])
        xz_plane2.set_facecolor((0, 0.5, 0.5, 0.5))
        ax.add_collection3d(xz_plane2)

    def plot_circle(center, radius):
        """theta = np.linspace(0, 2 * np.pi, 201)
        y = 10*np.cos(theta)
        z = 10*np.sin(theta)
        for i in range(18):
            phi = i*np.pi/9
            ax.plot(y*np.sin(phi)+10*np.sin(phi),
                    y*np.cos(phi)+10*np.cos(phi), z)"""
        
        p = Circle(center, radius)
        ax.add_patch(p)
        art3d.pathpatch_2d_to_3d(p, z=0, zdir="x")

    #         return ax

    # (seq_len, joints_num, 3)
    #print(joints.shape)
    #print(len(joints))
    data = joints.copy().reshape(len(joints), -1, 3)

    # preparation related to specific datasets
    if dataset == 'kit':
        data *= 0.003  # scale for visualization
    elif dataset == 'humanml':
        data *= 1.3  # scale for visualization
    elif dataset in ['humanact12', 'uestc']:
        data *= -1.5 # reverse axes, scale for visualization

    #print(data[0].shape)


    #print("figsize", figsize)
    fig = plt.figure(figsize=figsize)
    plt.tight_layout()
    ax = p3.Axes3D(fig)
    init()
    MINS = data.min(axis=0).min(axis=0)
    MAXS = data.max(axis=0).max(axis=0)
    #print("MINS MAXS", [MINS, MAXS])
    colors_blue = ["#4D84AA", "#5B9965", "#61CEB9", "#34C1E2", "#80B79A"]  # GT color
    colors_orange = ["#DD5A37", "#D69E00", "#B75A39", "#FF6D00", "#DDB50E"]  # Generation color
    colors = colors_orange
    if vis_mode == 'upper_body':  # lower body taken fixed to input motion
        colors[0] = colors_blue[0]
        colors[1] = colors_blue[1]
    elif vis_mode == 'gt':
        colors = colors_blue

    frame_number = data.shape[0]
    #print("frame_number", frame_number)

    height_offset = MINS[1]
    data[:, :, 1] -= height_offset
    trajec = data[:, 0, [0, 2]] #x and y of joint 0
    """
    print("data max 0", data[-1, :, [0, 2]][0].max())
    print("data max 0", data[-1, :, [0, 2]][0].min())
    print("data max 1", data[-1, :, [0, 2]][1].max())
    print("data max 1", data[-1, :, [0, 2]][1].min())
    """
    maxx = data[-1, :, [0, 2]][0].max()
    minx = data[-1, :, [0, 2]][0].min()
    maxz = data[-1, :, [0, 2]][1].max()
    minz = data[-1, :, [0, 2]][1].min()

    np.save(save_path+ "_ending_area.npy", np.array([maxx, minx, maxz, minz]))
    #print(trajec.shape)
    #print(trajec)
    #ax.scatter(trajec[:,0], trajec[:,1], np.array(120*[0]))
    

    data[..., 0] -= data[:, 0:1, 0]
    data[..., 2] -= data[:, 0:1, 2]

    #print("trajc shape", trajec.shape)

    def update(index):
        ax.clear()
        """
        print("update")
        ax.lines = []
        print("ax")
        ax.collections = []
        print("viewinit")
        """
        
        ax.view_init(elev=120, azim=-90)
        #ax.view_init(elev=180, azim=-90)
        ax.dist = 7.5
        #         ax =
        #print("plot bizarre")

        #print("sol", [MINS[0] - trajec[index, 0], MAXS[0] - trajec[index, 0], 0, MINS[2] - trajec[index, 1],
        #             MAXS[2] - trajec[index, 1]])
                     
        plot_xzPlane(MINS[0] - trajec[index, 0], MAXS[0] - trajec[index, 0], 0, MINS[2] - trajec[index, 1],
                     MAXS[2] - trajec[index, 1])
        
        #print("trajec", [trajec[-1,0] - trajec[index, 0], trajec[-1,0]+1 - trajec[index, 0], 0, trajec[-1,1] - trajec[index, 1], trajec[-1,1] + 1 - trajec[index, 1]])

        #plot_xzPlane2(trajec[-1,0] - 1/2 - trajec[index, 0], trajec[-1,0] + 1/2 - trajec[index, 0], 0, trajec[-1,1] - 1/2 - trajec[index, 1], trajec[-1,1] + 1/2 - trajec[index, 1])        
        
        """Important"""
        plot_xzPlane2(minx - trajec[index, 0], maxx - trajec[index, 0], 0, minz - trajec[index, 1], maxz - trajec[index, 1])

        #plot_circle((minx - trajec[index, 0], maxx - trajec[index, 0]), 2)
        """_________"""

        #x_grid,y_grid,z_grid = data_for_cylinder_along_z(MAXS[0] + trajec[index, 0], MAXS[2] + trajec[index, 1], 1, 1)
        #ax.plot_surface(x_grid, y_grid, z_grid, alpha=0.5)



        #plot_xzPlane2(MINS[0], MAXS[0], 0, MINS[2], MAXS[2])

        
        #         ax.scatter(dataset[index, :22, 0], dataset[index, :22, 1], dataset[index, :22, 2], color='black', s=3)

        # if index > 1:
        #     ax.plot3D(trajec[:index, 0] - trajec[index, 0], np.zeros_like(trajec[:index, 0]),
        #               trajec[:index, 1] - trajec[index, 1], linewidth=1.0,
        #               color='blue')
        # #             ax = plot_xzPlane(ax, MINS[0], MAXS[0], 0, MINS[2], MAXS[2])


        #Compute average position
        
        avx = np.average(data[index, :, [0, 2]][0])
        avy = np.average(data[index, :, [0, 2]][1])
        centerx = (minx + maxx)/2
        centery = (minz + maxz)/2


        print("av pos x", avx)
        print("av pos y", avy)

        print(trajec[-1, 0])
        #dist = ((minx-avx)**2 + (minz-avy)**2)**(1/2)
        dist = ((trajec[-1, 0]-avx)**2 + (trajec[-1, 1]-avy)**2)**(1/2)
        
        print("dist to ending", dist)
        #ax.text(x=0., y=0., z=0., s=str(dist))



        used_colors = colors_blue if index in gt_frames else colors
        for i, (chain, color) in enumerate(zip(kinematic_tree, used_colors)):
            #print("boucle")
            if i < 5:
                linewidth = 4.0
            else:
                linewidth = 2.0
            ax.plot3D(data[index, chain, 0], data[index, chain, 1], data[index, chain, 2], linewidth=linewidth,
                      color=color)
            """ax.plot3D(data[-1, chain, 0] - trajec[index, 0], data[-1, chain, 1]- trajec[index, 1] , data[-1, chain, 2], linewidth=linewidth,
                      color=color)"""
        #         print(trajec[:index, 0].shape)
            #ax.plot3D(data[-1, chain, 1], data[-1, chain, 1], data[-1, chain, 2], linewidth=4.0,color=color)
        #print(data[index, chain, :])
        #plot_xzPlane(data[index, chain, 0][0] - trajec[index, 0], data[index, chain, 0][-1] - trajec[index, 0], 0, data[index, chain, 2][0] - trajec[index, 1],
        #             data[index, chain, 2][-1] - trajec[index, 1])
        #plot_xzPlane_end(data[-1, 7, 0] - trajec[index, 0], data[-1, 10, 0] - trajec[index, 0], 0, data[-1, 8, 1] - trajec[index, 1], data[-1, 11, 0] - trajec[index, 1])
        

        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_zticklabels([])

    ani = FuncAnimation(fig, update, frames=frame_number, interval=1000 / fps, repeat=False)

    #writer = FFMpegFileWriter(fps=fps)
    ani.save(save_path, fps=fps)
    # ani = FuncAnimation(fig, update, frames=frame_number, interval=1000 / fps, repeat=False, init_func=init)
    #ani.save(save_path, writer='pillow', fps=1000 / fps)

    

    plt.close()

In [3]:
import numpy as np

# Define a kinematic tree for the skeletal struture
kit_kinematic_chain = [[0, 11, 12, 13, 14, 15], [0, 16, 17, 18, 19, 20], [0, 1, 2, 3, 4], [3, 5, 6, 7], [3, 8, 9, 10]]

kit_raw_offsets = np.array(
    [
        [0, 0, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [1, 0, 0],
        [0, -1, 0],
        [0, -1, 0],
        [-1, 0, 0],
        [0, -1, 0],
        [0, -1, 0],
        [1, 0, 0],
        [0, -1, 0],
        [0, -1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [-1, 0, 0],
        [0, -1, 0],
        [0, -1, 0],
        [0, 0, 1],
        [0, 0, 1]
    ]
)

t2m_raw_offsets = np.array([[0,0,0],
                           [1,0,0],
                           [-1,0,0],
                           [0,1,0],
                           [0,-1,0],
                           [0,-1,0],
                           [0,1,0],
                           [0,-1,0],
                           [0,-1,0],
                           [0,1,0],
                           [0,0,1],
                           [0,0,1],
                           [0,1,0],
                           [1,0,0],
                           [-1,0,0],
                           [0,0,1],
                           [0,-1,0],
                           [0,-1,0],
                           [0,-1,0],
                           [0,-1,0],
                           [0,-1,0],
                           [0,-1,0]])

t2m_kinematic_chain = [[0, 2, 5, 8, 11], [0, 1, 4, 7, 10], [0, 3, 6, 9, 12, 15], [9, 14, 17, 19, 21], [9, 13, 16, 18, 20]]
t2m_left_hand_chain = [[20, 22, 23, 24], [20, 34, 35, 36], [20, 25, 26, 27], [20, 31, 32, 33], [20, 28, 29, 30]]
t2m_right_hand_chain = [[21, 43, 44, 45], [21, 46, 47, 48], [21, 40, 41, 42], [21, 37, 38, 39], [21, 49, 50, 51]]


kit_tgt_skel_id = '03950'

t2m_tgt_skel_id = '000021'

In [9]:
import numpy as np

def load_motion_from_dataset(name):
    data = np.load(r'HumanML3D\new_joints\\' + name + '.npy', allow_pickle=True)
    text = np.loadtxt(r'HumanML3D\texts\\' + name + '.txt', dtype='str', comments="#", delimiter=",", usecols=0)[0]
    print(text)
    print(data.shape)
    #print(data.shape)

    motion = data

    return text, motion

In [7]:
import os

fps=20
dataset = "humanml"
skeleton = kit_kinematic_chain if dataset == 'kit' else t2m_kinematic_chain

def render_motion(caption, motion, fps, name, dataset, skeleton):
    length = motion.shape[0]
    motion = motion.transpose(0, 1, 2)[:length]
    #print("motion shape render", motion.shape)
    save_file = name + 'hey3.gif'

    animation_save_path = os.path.join("présentation", save_file)
    plot_3d_motion(animation_save_path, skeleton, motion, dataset=dataset, title=caption, fps=fps)

In [10]:
dir_path = r"HumanML3D\new_joints\\"

f = ['000004']

# list to store files
res = []

# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)) and path[:-4] in f:
        res.append(path[:-4])
        text, motion = load_motion_from_dataset(path[:-4])
        #render_motion(text, motion, fps, path[:-4], dataset, skeleton)

a man is pretending to be a chicken. constantly pecking at the ground and waving his arms like a chicken.
(199, 22, 3)


In [10]:
dir_path = r"HumanML3D\new_joints\\"


# list to store files
res = []

# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        res.append(path[:-4])
        text, motion = load_motion_from_dataset(path[:-4])
        render_motion(text, motion, fps, path[:-4], dataset, skeleton)

a man kicks something or someone with his left leg.
av pos x 0.010134567
av pos y -0.020531563
0.028810577
dist to ending 0.05348528718977251
av pos x 0.010134567
av pos y -0.020531563
0.028810577
dist to ending 0.05348528718977251
av pos x 0.009783357
av pos y -0.015733654
0.028810577
dist to ending 0.04915290208515419
av pos x 0.009383931
av pos y -0.0062887785
0.028810577
dist to ending 0.04079797727977074
av pos x 0.008965335
av pos y 0.003780788
0.028810577
dist to ending 0.03255457815869411
av pos x 0.008071814
av pos y 0.01170648
0.028810577
dist to ending 0.027382725231354068
av pos x 0.0061652716
av pos y 0.019896837
0.028810577
dist to ending 0.024631514261494066


C:\Users\mathi\AppData\Local\Temp\ipykernel_20476\1523405687.py:89: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  ax = p3.Axes3D(fig)
C:\Users\mathi\AppData\Local\Temp\ipykernel_20476\1523405687.py:40: MatplotlibDeprecationWarning: The 'b' parameter of grid() has been renamed 'visible' since Matplotlib 3.5; support for the old name will be dropped two minor releases later.
  ax.grid(b=False)


av pos x 0.003411705
av pos y 0.03319276
0.028810577
dist to ending 0.02565352560160006
av pos x 0.00033537232
av pos y 0.053242944
0.028810577
dist to ending 0.03701938962907663
av pos x -0.002968344
av pos y 0.076029524
0.028810577
dist to ending 0.05627429883985475
av pos x -0.006331588
av pos y 0.09524884
0.028810577
dist to ending 0.07447438038615914
av pos x -0.012667672
av pos y 0.10144124
0.028810577
dist to ending 0.08296662029752915
av pos x -0.022491619
av pos y 0.09398702
0.028810577
dist to ending 0.08233627414107086
av pos x -0.03477638
av pos y 0.08039766
0.028810577
dist to ending 0.08139417664491089
av pos x -0.04793016
av pos y 0.07195134
0.028810577
dist to ending 0.08765767056347616
av pos x -0.062685676
av pos y 0.070786685
0.028810577
dist to ending 0.10034424843299743
av pos x -0.077742144
av pos y 0.07146934
0.028810577
dist to ending 0.11448843718601248
av pos x -0.08953426
av pos y 0.06844214
0.028810577
dist to ending 0.1245600781081115
av pos x -0.09675362
a

C:\Users\mathi\AppData\Local\Temp\ipykernel_20476\1523405687.py:89: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  ax = p3.Axes3D(fig)
C:\Users\mathi\AppData\Local\Temp\ipykernel_20476\1523405687.py:40: MatplotlibDeprecationWarning: The 'b' parameter of grid() has been renamed 'visible' since Matplotlib 3.5; support for the old name will be dropped two minor releases later.
  ax.grid(b=False)


av pos x 0.01667814
av pos y -0.21964978
0.02881086
dist to ending 0.2619248608393738
av pos x 0.007891888
av pos y -0.23421082
0.02881086
dist to ending 0.27699580420029585
av pos x 0.010547164
av pos y -0.2085345
0.02881086
dist to ending 0.2511932907257932
av pos x 0.0122944275
av pos y -0.23454684
0.02881086
dist to ending 0.2770335699304883
av pos x -0.005102545
av pos y -0.24516326
0.02881086
dist to ending 0.2891528677078786
av pos x -0.014329133
av pos y -0.24931173
0.02881086
dist to ending 0.29448267749192314
av pos x -0.013984415
av pos y -0.2457508
0.02881086
dist to ending 0.2909097501191769
av pos x -0.023875969
av pos y -0.25915554
0.02881086
dist to ending 0.3057235937598982
av pos x -0.01407547
av pos y -0.24945106
0.02881086
dist to ending 0.29458349461202593
av pos x -0.011333853
av pos y -0.24828444
0.02881086
dist to ending 0.29304117963031523
av pos x -0.007622762
av pos y -0.25421548
0.02881086
dist to ending 0.2984416729126766
av pos x 0.007788616
av pos y -0.24

...

C:\Users\mathi\AppData\Local\Temp\ipykernel_20476\1523405687.py:89: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  ax = p3.Axes3D(fig)
C:\Users\mathi\AppData\Local\Temp\ipykernel_20476\1523405687.py:40: MatplotlibDeprecationWarning: The 'b' parameter of grid() has been renamed 'visible' since Matplotlib 3.5; support for the old name will be dropped two minor releases later.
  ax.grid(b=False)


av pos x 0.0024817912
av pos y -0.045437366
-1.944909
dist to ending 3.0086557173124526
av pos x 0.002332327
av pos y -0.04557346
-1.944909
dist to ending 3.0086627385364717
av pos x 0.0022766537
av pos y -0.045684274
-1.944909
dist to ending 3.008711223229235
av pos x 0.002271621
av pos y -0.045640524
-1.944909
dist to ending 3.0086745401338915
av pos x 0.0024091683
av pos y -0.045491353
-1.944909
dist to ending 3.0086499830556277
av pos x 0.0025539899
av pos y -0.045297507
-1.944909
dist to ending 3.0085959036417025
av pos x 0.0027032995
av pos y -0.045121297
-1.944909
dist to ending 3.0085581178848546
av pos x 0.0028509675
av pos y -0.0448598
-1.944909
dist to ending 3.0084545814457586
av pos x 0.0029215568
av pos y -0.04458281
-1.944909
dist to ending 3.008289139553974
av pos x 0.0028738691
av pos y -0.044349167
-1.944909
dist to ending 3.008080207955524
av pos x 0.0027584014
av pos y -0.04408921
-1.944909
dist to ending 3.007807372910314
av pos x 0.0024940318
av pos y -0.043905523

C:\Users\mathi\AppData\Local\Temp\ipykernel_20476\1523405687.py:89: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  ax = p3.Axes3D(fig)
C:\Users\mathi\AppData\Local\Temp\ipykernel_20476\1523405687.py:40: MatplotlibDeprecationWarning: The 'b' parameter of grid() has been renamed 'visible' since Matplotlib 3.5; support for the old name will be dropped two minor releases later.
  ax.grid(b=False)


av pos x 0.0042273025
av pos y 0.021159597
-0.45322466
dist to ending 1.4824249567827947
av pos x 0.0011249862
av pos y 0.021107439
-0.45322466
dist to ending 1.4815202645733163
av pos x 0.00021080673
av pos y 0.023052644
-0.45322466
dist to ending 1.4793884164912479
av pos x 0.000814572
av pos y 0.026620416
-0.45322466
dist to ending 1.476178422683336
av pos x 0.0010769273
av pos y 0.03150125
-0.45322466
dist to ending 1.4716158726356225
av pos x 0.0011065927
av pos y 0.039356254
-0.45322466
dist to ending 1.4641558067253162
av pos x 0.003113557
av pos y 0.041982654
-0.45322466
dist to ending 1.4622843387366677
av pos x 0.007518129
av pos y 0.037324417
-0.45322466
dist to ending 1.4680870069655756
av pos x 0.015274327
av pos y 0.030441957
-0.45322466
dist to ending 1.477065067461597
av pos x 0.02340708
av pos y 0.02643293
-0.45322466
dist to ending 1.483460732567476
av pos x 0.027967524
av pos y 0.026281906
-0.45322466
dist to ending 1.4850750454241444
av pos x 0.02799297
av pos y 0.0

KeyboardInterrupt: 

In [66]:
import numpy as np

data = np.load('results/samples_humanml_trans_enc_512_000200000_seed10_a_man_runs_in_a_straight_line_for_two_meter/results.npy', allow_pickle=True)
#data = np.load(r'C:\Users\mathi\Desktop\10.2\HumanML3D\new_joints\000000.npy', allow_pickle=True)

print(data)

all_motions = data.tolist()['motion']
all_lengths = data.tolist()['lengths']
all_text = data.tolist()['text']
num_samples = data.tolist()['num_samples']
num_repetitions = data.tolist()['num_repetitions']

{'motion': array([[[[ 0.00000000e+00,  1.78163517e-02,  3.31552885e-02, ...,
          -1.22477695e-01, -1.21369131e-01, -1.20208539e-01],
         [ 9.99614716e-01,  9.76285756e-01,  9.34447587e-01, ...,
           1.07769799e+00,  1.07767332e+00,  1.07838929e+00],
         [ 0.00000000e+00,  1.34657696e-01,  2.65752137e-01, ...,
           4.21168232e+00,  4.21280861e+00,  4.21388960e+00]],

        [[ 4.82580736e-02,  7.34596550e-02,  9.40031931e-02, ...,
          -7.06552938e-02, -6.96103871e-02, -6.84421360e-02],
         [ 8.94041896e-01,  8.72595131e-01,  8.29935789e-01, ...,
           9.55998063e-01,  9.55975056e-01,  9.56729770e-01],
         [-4.54240777e-02,  9.42838117e-02,  2.37734571e-01, ...,
           4.22668171e+00,  4.22773218e+00,  4.22873688e+00]],

        [[-7.25209042e-02, -5.08648902e-02, -3.30531336e-02, ...,
          -1.95344150e-01, -1.94103837e-01, -1.92737326e-01],
         [ 8.78292382e-01,  8.53580356e-01,  8.11162591e-01, ...,
           9.56871927e-

In [ ]:
#Dataloader avec coordonnées finales



In [76]:
data.tolist().keys()

dict_keys(['motion', 'text', 'lengths', 'num_samples', 'num_repetitions'])

In [67]:
print('num_repetitions', data.tolist()['num_repetitions'])
print('num_samples', data.tolist()['num_samples'])
print('text', data.tolist()['text'])
print('lengths', data.tolist()['lengths'])
print('motion', data.tolist()['motion'].shape)

num_repetitions 3
num_samples 1
text ['a man runs in a straight line for two meter.', 'a man runs in a straight line for two meter.', 'a man runs in a straight line for two meter.']
lengths [120 120 120]
motion (3, 22, 3, 120)


In [78]:
joints = data.tolist()['motion']

dta = joints.copy().reshape(len(joints), -1, 3)
dta[:, 0, [0, 2]]

array([[ 0.        ,  0.03315529],
       [ 0.        , -0.03028736],
       [ 0.        , -0.02521757]], dtype=float32)

In [84]:
import os

fps=20
dataset = "humanml"
skeleton = kit_kinematic_chain if dataset == 'kit' else t2m_kinematic_chain


for sample_i in range(0, num_samples):
    for rep_i in range(0, num_repetitions):
        caption = all_text[rep_i*num_samples + sample_i]
        length = all_lengths[rep_i*num_samples + sample_i]
        print(length)
        motion = all_motions[rep_i*num_samples + sample_i].transpose(2, 0, 1)[:length]
        print(motion.shape)
        save_file = str(sample_i) + str(rep_i) + 'hey.mp4'

        animation_save_path = os.path.join("out", save_file)
        plot_3d_motion(animation_save_path, skeleton, motion, dataset=dataset, title=caption, fps=fps)
        # Credit for visualization: https://github.com/EricGuo5513/text-to-motion

120
(120, 22, 3)
(120, 22, 3)
120
sol [-1.0299995, 0.4847581, 0, -0.78575724, 5.8627057]
sol [-1.0299995, 0.4847581, 0, -0.78575724, 5.8627057]
sol [-1.0531608, 0.46159685, 0, -0.9608122, 5.6876507]
sol [-1.0731014, 0.44165623, 0, -1.131235, 5.517228]
sol [-1.0883111, 0.4264466, 0, -1.298008, 5.350455]
sol [-1.0975853, 0.41717225, 0, -1.4936897, 5.154773]


C:\Users\mathi\AppData\Local\Temp\ipykernel_4376\3445225925.py:97: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  ax = p3.Axes3D(fig)
C:\Users\mathi\AppData\Local\Temp\ipykernel_4376\3445225925.py:40: MatplotlibDeprecationWarning: The 'b' parameter of grid() has been renamed 'visible' since Matplotlib 3.5; support for the old name will be dropped two minor releases later.
  ax.grid(b=False)


sol [-1.0938766, 0.420881, 0, -1.7205181, 4.9279447]
sol [-1.0845206, 0.430237, 0, -1.9576583, 4.6908045]
sol [-1.0757226, 0.43903503, 0, -2.1957533, 4.4527097]
sol [-1.0686067, 0.44615087, 0, -2.4366636, 4.211799]
sol [-1.0623665, 0.4523911, 0, -2.6737626, 3.9747005]
sol [-1.0545745, 0.4601831, 0, -2.8865354, 3.7619276]
sol [-1.0437988, 0.47095877, 0, -3.0806127, 3.5678504]
sol [-1.0260388, 0.4887189, 0, -3.2822113, 3.3662517]
sol [-0.9997472, 0.51501036, 0, -3.5020869, 3.1463761]
sol [-0.971534, 0.54322356, 0, -3.7245653, 2.9238977]
sol [-0.9437333, 0.5710243, 0, -3.946691, 2.701772]
sol [-0.9161729, 0.5985847, 0, -4.169322, 2.4791408]
sol [-0.88522094, 0.6295366, 0, -4.378451, 2.2700121]
sol [-0.8535987, 0.6611589, 0, -4.5586777, 2.0897853]
sol [-0.8283845, 0.68637305, 0, -4.7199707, 1.9284921]
sol [-0.80960643, 0.7051512, 0, -4.8655934, 1.7828693]
sol [-0.7935673, 0.72119033, 0, -4.9950724, 1.6533904]
sol [-0.77789164, 0.736866, 0, -5.116467, 1.5319958]
sol [-0.7610941, 0.75366354,

C:\Users\mathi\AppData\Local\Temp\ipykernel_4376\3445225925.py:97: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  ax = p3.Axes3D(fig)
C:\Users\mathi\AppData\Local\Temp\ipykernel_4376\3445225925.py:40: MatplotlibDeprecationWarning: The 'b' parameter of grid() has been renamed 'visible' since Matplotlib 3.5; support for the old name will be dropped two minor releases later.
  ax.grid(b=False)


sol [-1.4320449, 0.39315155, 0, -1.0916489, 7.2860413]
sol [-1.4154875, 0.40970886, 0, -1.2212095, 7.156481]
sol [-1.3964555, 0.42874092, 0, -1.3648683, 7.012822]
sol [-1.375373, 0.44982338, 0, -1.5188439, 6.858846]
sol [-1.3541085, 0.471088, 0, -1.6733408, 6.7043495]
sol [-1.3356667, 0.4895297, 0, -1.8254117, 6.5522785]
sol [-1.3200334, 0.505163, 0, -1.9717108, 6.405979]
sol [-1.3076276, 0.5175689, 0, -2.101491, 6.2761993]
sol [-1.2987587, 0.52643764, 0, -2.2191448, 6.1585455]
sol [-1.2910109, 0.5341855, 0, -2.3372571, 6.040433]
sol [-1.2823647, 0.54283166, 0, -2.4627156, 5.9149747]
sol [-1.2711773, 0.5540192, 0, -2.598772, 5.7789183]
sol [-1.2585725, 0.566624, 0, -2.7366796, 5.6410108]
sol [-1.24603, 0.5791664, 0, -2.8725238, 5.505166]
sol [-1.2349355, 0.59026086, 0, -3.002872, 5.374818]
sol [-1.2264718, 0.5987246, 0, -3.1172175, 5.2604723]
sol [-1.2187119, 0.60648453, 0, -3.2165446, 5.1611457]
sol [-1.210808, 0.61438835, 0, -3.3110385, 5.066652]
sol [-1.2028358, 0.62236065, 0, -3.40

C:\Users\mathi\AppData\Local\Temp\ipykernel_4376\3445225925.py:97: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  ax = p3.Axes3D(fig)
C:\Users\mathi\AppData\Local\Temp\ipykernel_4376\3445225925.py:40: MatplotlibDeprecationWarning: The 'b' parameter of grid() has been renamed 'visible' since Matplotlib 3.5; support for the old name will be dropped two minor releases later.
  ax.grid(b=False)


sol [-5.667994, 0.6152994, 0, -1.0062473, 8.401044]
sol [-5.650268, 0.63302505, 0, -1.2121079, 8.195183]
sol [-5.6328793, 0.6504141, 0, -1.3924551, 8.014835]
sol [-5.624546, 0.6587472, 0, -1.561341, 7.8459496]
sol [-5.6257205, 0.6575727, 0, -1.7318176, 7.675473]
sol [-5.631486, 0.6518074, 0, -1.9044057, 7.502885]
sol [-5.640574, 0.6427192, 0, -2.0729117, 7.334379]
sol [-5.6494255, 0.6338675, 0, -2.2265391, 7.180752]
sol [-5.657687, 0.6256062, 0, -2.3629847, 7.0443063]
sol [-5.6641135, 0.61917955, 0, -2.4931345, 6.914156]
sol [-5.667654, 0.6156392, 0, -2.6237311, 6.78356]
sol [-5.6715636, 0.6117296, 0, -2.747683, 6.659608]
sol [-5.674248, 0.6090448, 0, -2.8587813, 6.5485096]
sol [-5.6784606, 0.6048325, 0, -2.954558, 6.452733]
sol [-5.6831527, 0.6001405, 0, -3.0361056, 6.3711853]
sol [-5.686224, 0.59706926, 0, -3.1037307, 6.3035603]
sol [-5.6863956, 0.5968976, 0, -3.1577716, 6.2495193]
sol [-5.6830864, 0.60020703, 0, -3.1977363, 6.2095547]
sol [-5.6767826, 0.6065105, 0, -3.225719, 6.1815

In [80]:
traj = np.load("trajec.npy")
print(traj[:, 1])

[0.         0.19973965 0.41046283 0.6230548  0.82891554 1.0092627
 1.1781486  1.3486252  1.5212133  1.6897193  1.8433466  1.9797922
 2.1099422  2.2405386  2.3644905  2.475589   2.5713654  2.6529133
 2.7205381  2.7745793  2.814544   2.8425264  2.8601482  2.8700168
 2.8757854  2.8807132  2.8871236  2.8960812  2.9075856  2.9214275
 2.9375598  2.9559777  2.976092   2.9976022  3.0203192  3.0437157
 3.0673816  3.0917132  3.1188104  3.1509063  3.185992   3.2245862
 3.2673116  3.314565   3.3618994  3.408117   3.4554822  3.5110016
 3.5809019  3.6716845  3.7828705  3.9045548  4.029339   4.150626
 4.2822075  4.433803   4.5896406  4.7425346  4.8788686  5.0066094
 5.1434364  5.2872705  5.435687   5.5854344  5.7258945  5.8649564
 6.015095   6.18081    6.349004   6.5178657  6.680396   6.8218927
 6.954991   7.0863476  7.213968   7.331926   7.4363313  7.5102286
 7.563014   7.60305    7.6353135  7.6646967  7.6931853  7.718197
 7.7402005  7.7593074  7.775122   7.78847    7.800054   7.8102736
 7.8193603  

In [81]:
plt.clf()
plt.plot(traj[:, 0], traj[:, 1])
plt.savefig("fig.png")